# Blocks and Other Pyomo Best Practices

## Learning Objectives

1. Practice using blocks to organize Pyomo models with a hierachical structure (e.g., two-stage stochastic program)
2. Learn best practices for organizing Pyomo models and data

## Blocks in Pyomo

Reference: [Chapter 8: Stuctured Modeling with Blocks](https://link.springer.com/chapter/10.1007/978-3-030-68928-5_8), *Pyomo -- Optimization Modeling in Python*, Bynum et al. (2021).

Blocks provide a convenient way to express hierachically-structured models in Pyomo. As an example, consider the special structure in the *Farmer's* example for [](./SP.ipynb).

Asking ChatGPT to rewrite our model from [](./SP.ipynb) with blocks gives the following code (which was then slightly modified):

In [1]:
from pyomo.environ import ConcreteModel, Var, Objective, Block, Constraint, NonNegativeReals, summation, SolverFactory, minimize

def build_sp_model(yields):
    '''
    Rewritten version of the stochastic programming model using blocks.
    
    Arguments:
        yields: Yield information as a list, following the rank [wheat, corn, beets]
        
    Return: 
        model: farmer problem model using blocks
    '''
    
    # Model
    model = ConcreteModel()

    # Sets
    all_crops = ["WHEAT", "CORN", "BEETS"]
    purchase_crops = ["WHEAT", "CORN"]
    sell_crops = ["WHEAT", "CORN", "BEETS_FAVORABLE", "BEETS_UNFAVORABLE"]
    scenarios = ["ABOVE", "AVERAGE", "BELOW"]

    # First-stage decision: how much to land to dedicate to each crop
    model.X = Var(all_crops, within=NonNegativeReals)

    # Define a block for each scenario
    def scenario_block_rule(b, scenario):
        # Second-stage decision variables
        b.Y = Var(purchase_crops, within=NonNegativeReals)  # How much to purchase in this scenario
        b.W = Var(sell_crops, within=NonNegativeReals)  # How much to sell in this scenario
        
        # Purchase cost and sales revenue for this scenario
        if scenario == "ABOVE":
            b.purchase_cost = 238 * b.Y["WHEAT"] + 210 * b.Y["CORN"]
            b.sales_revenue = (
                170 * b.W["WHEAT"] + 150 * b.W["CORN"] 
                + 36 * b.W["BEETS_FAVORABLE"] + 10 * b.W["BEETS_UNFAVORABLE"]
            )
        elif scenario == "AVERAGE":
            b.purchase_cost = 238 * b.Y["WHEAT"] + 210 * b.Y["CORN"]
            b.sales_revenue = (
                170 * b.W["WHEAT"] + 150 * b.W["CORN"] 
                + 36 * b.W["BEETS_FAVORABLE"] + 10 * b.W["BEETS_UNFAVORABLE"]
            )
        else:  # BELOW
            b.purchase_cost = 238 * b.Y["WHEAT"] + 210 * b.Y["CORN"]
            b.sales_revenue = (
                170 * b.W["WHEAT"] + 150 * b.W["CORN"] 
                + 36 * b.W["BEETS_FAVORABLE"] + 10 * b.W["BEETS_UNFAVORABLE"]
            )

        # Scenario constraints
        if scenario == "ABOVE":
            b.wheat_constraint = Constraint(expr=yields[0] * 1.2 * model.X["WHEAT"] + b.Y["WHEAT"] - b.W["WHEAT"] >= 200)
            b.corn_constraint = Constraint(expr=yields[1] * 1.2 * model.X["CORN"] + b.Y["CORN"] - b.W["CORN"] >= 240)
            b.beets_constraint = Constraint(expr=yields[2] * 1.2 * model.X["BEETS"] 
                                            - b.W["BEETS_FAVORABLE"] - b.W["BEETS_UNFAVORABLE"] >= 0)
        elif scenario == "AVERAGE":
            b.wheat_constraint = Constraint(expr=yields[0] * model.X["WHEAT"] + b.Y["WHEAT"] - b.W["WHEAT"] >= 200)
            b.corn_constraint = Constraint(expr=yields[1] * model.X["CORN"] + b.Y["CORN"] - b.W["CORN"] >= 240)
            b.beets_constraint = Constraint(expr=yields[2] * model.X["BEETS"] 
                                            - b.W["BEETS_FAVORABLE"] - b.W["BEETS_UNFAVORABLE"] >= 0)
        else:  # BELOW
            b.wheat_constraint = Constraint(expr=yields[0] * 0.8 * model.X["WHEAT"] + b.Y["WHEAT"] - b.W["WHEAT"] >= 200)
            b.corn_constraint = Constraint(expr=yields[1] * 0.8 * model.X["CORN"] + b.Y["CORN"] - b.W["CORN"] >= 240)
            b.beets_constraint = Constraint(expr=yields[2] * 0.8 * model.X["BEETS"] 
                                            - b.W["BEETS_FAVORABLE"] - b.W["BEETS_UNFAVORABLE"] >= 0)

        # Set upper bounds for BEETS_FAVORABLE
        b.W["BEETS_FAVORABLE"].setub(6000)

    # Create blocks for each scenario
    model.scenarios = Block(scenarios, rule=scenario_block_rule)

    # Objective function
    def objective_rule(m):
        planting_cost = 150 * m.X["WHEAT"] + 230 * m.X["CORN"] + 260 * m.X["BEETS"]
        expected_purchase_cost = (1/3) * sum(m.scenarios[sc].purchase_cost for sc in scenarios)
        expected_sales_revenue = (1/3) * sum(m.scenarios[sc].sales_revenue for sc in scenarios)
        return planting_cost + expected_purchase_cost - expected_sales_revenue

    model.OBJ = Objective(rule=objective_rule, sense=minimize)

    # First-stage constraint: total area allocated to crops should not exceed 500
    model.total_land_constraint = Constraint(expr=summation(model.X) <= 500)

    return model

yields = [2.5, 3.0, 20.0]

model = build_sp_model(yields)
solver = SolverFactory('cbc')
solver.solve(model)

# Display the results
print("Planting decisions:")
for crop in model.X:
    print(f"{crop}: {model.X[crop]()}")

Planting decisions:
WHEAT: 170.0
CORN: 80.0
BEETS: 250.0


We got the same answer as [](./SP.ipynb)!

## Seperate the Data from the Model

The above blocks example is not really a big improvement from our alternate implementation in [](./SP.ipynb).

In the above code (from ChatGPT), we see:
* Data for each scenario are hardcoded into the model
* If statements are used to toggle the correct constraints for each scenario

The above code is not general purpose; updating it to use alternate scenario data or consider additional crops would take a lot of manual effort. It would be easy to make mistakes.

Thus, it is **best practice** to always **seperate your specific problem data** from the **general mathematical model**. Let's see an example.

### Pandas DataFrames

Let's use a pandas dataframe to store our problem data.

In [2]:
import pandas as pd

nominal_data = pd.read_csv("../data/farmers.csv")

nominal_data.head()

,Unnamed: 0,Wheat,Corn,Beats,Units
0,Yield,2.5,3.0,20.0,T/acre
1,Planting Cost,150.0,230.0,260.0,USD/acre
2,Favorable Selling Price,170.0,150.0,36.0,USD/T
3,Unfavorable Selling Price,NaN,NaN,10.0,USD/T
4,Purchase Price,238.0,210.0,NaN,USD/T


This looks great. But it is best practice to have the rows be instances of data and the columns to be the types of data. We need to transpose the CSV file. Also, let's drop the units for simplicity. ChatGPT is actually really helpful for transposing the CSV file!

In [3]:
nominal_data = pd.read_csv("../data/farmers2.csv")
nominal_data.head()

,index,Yield,Planting Cost,Favorable Selling Price,Unfavorable Selling Price,Purchase Price,Minimum Requirement,Maximum Favorable Production
0,Wheat,2.5,150,170,NaN,238,200,NaN
1,Corn,3,230,150,NaN,210,240,NaN
2,Beats,20,260,36,10.0,NaN,NaN,6000
3,Units,T/acre,USD/acre,USD/T,NaN,USD/T,T,T


Now let's drop the units for simplicity.

In [4]:
nominal_data = nominal_data.set_index("index")
nominal_data.head()

,Yield,Planting Cost,Favorable Selling Price,Unfavorable Selling Price,Purchase Price,Minimum Requirement,Maximum Favorable Production
index,,,,,,,
Wheat,2.5,150,170,NaN,238,200,NaN
Corn,3,230,150,NaN,210,240,NaN
Beats,20,260,36,10.0,NaN,NaN,6000
Units,T/acre,USD/acre,USD/T,NaN,USD/T,T,T


In [5]:
nominal_data.drop("Units", inplace=True)
nominal_data.head()


,Yield,Planting Cost,Favorable Selling Price,Unfavorable Selling Price,Purchase Price,Minimum Requirement,Maximum Favorable Production
index,,,,,,,
Wheat,2.5,150,170,NaN,238,200,NaN
Corn,3,230,150,NaN,210,240,NaN
Beats,20,260,36,10.0,NaN,NaN,6000


Finally, we need to convert the data entries from strings to numbers.

In [6]:
# Convert all elements to numbers
nominal_data = nominal_data.map(lambda x: pd.to_numeric(x, errors='coerce'))

In [7]:
nominal_data.head()

,Yield,Planting Cost,Favorable Selling Price,Unfavorable Selling Price,Purchase Price,Minimum Requirement,Maximum Favorable Production
index,,,,,,,
Wheat,2.5,150,170,NaN,238.0,200.0,NaN
Corn,3.0,230,150,NaN,210.0,240.0,NaN
Beats,20.0,260,36,10.0,NaN,NaN,6000.0


Finally, we should remove the NaN values and add columns with booleans.

In [8]:
nominal_data['Enforce Max Production'] = False
nominal_data['Enforce Min Requirement'] = False
nominal_data['Allow Purchases'] = True

nominal_data.head()

,Yield,Planting Cost,Favorable Selling Price,Unfavorable Selling Price,Purchase Price,Minimum Requirement,Maximum Favorable Production,Enforce Max Production,Enforce Min Requirement,Allow Purchases
index,,,,,,,,,,
Wheat,2.5,150,170,NaN,238.0,200.0,NaN,False,False,True
Corn,3.0,230,150,NaN,210.0,240.0,NaN,False,False,True
Beats,20.0,260,36,10.0,NaN,NaN,6000.0,False,False,True


In [9]:
import numpy as np

for i, (index, row) in enumerate(nominal_data.iterrows()):

    # print(row)

    if not np.isnan(row['Maximum Favorable Production']):
        nominal_data.at[index, 'Enforce Max Production'] = True
    else:
        nominal_data.at[index, 'Maximum Favorable Production'] = np.inf
        nominal_data.at[index, 'Enforce Max Production'] = False

    if not np.isnan(row['Minimum Requirement']):
        nominal_data.at[index, 'Enforce Min Requirement'] = True
    else:
        nominal_data.at[index, 'Minimum Requirement'] = 0
        nominal_data.at[index, 'Enforce Min Requirement'] = False
    
    if np.isnan(row['Purchase Price']):
        nominal_data.at[index, 'Allow Purchases'] = False
        nominal_data.at[index, 'Purchase Price'] = 0
    else:
        nominal_data.at[index, 'Allow Purchases'] = True

    if np.isnan(row['Unfavorable Selling Price']):
        nominal_data.at[index, 'Unfavorable Selling Price'] = 0


nominal_data.head()

,Yield,Planting Cost,Favorable Selling Price,Unfavorable Selling Price,Purchase Price,Minimum Requirement,Maximum Favorable Production,Enforce Max Production,Enforce Min Requirement,Allow Purchases
index,,,,,,,,,,
Wheat,2.5,150,170,0.0,238.0,200.0,inf,False,True,True
Corn,3.0,230,150,0.0,210.0,240.0,inf,False,True,True
Beats,20.0,260,36,10.0,0.0,0.0,6000.0,True,False,False


### Single Scenario Optimization Problem

In [10]:
from pyomo.environ import Set, Param, Constraint, maximize
import numpy as np

def build_deterministic_model(nominal_data, m=None, skip_objective=False):

    # Create concrete model (if not provided)
    if m is None:
        m = ConcreteModel()

    # Sets
    crops = nominal_data.index.to_list()
    m.CROPS = Set(initialize=crops)

    m.max_area = 500

    # Parameters
    m.cost = Param(m.CROPS, initialize=nominal_data["Planting Cost"], within=NonNegativeReals)
    m.sell_price_favorable = Param(m.CROPS, initialize=nominal_data["Favorable Selling Price"], within=NonNegativeReals)
    m.sell_price_unfavorable = Param(m.CROPS, initialize=nominal_data["Unfavorable Selling Price"], within=NonNegativeReals)
    m.purchase_price = Param(m.CROPS, initialize=nominal_data["Purchase Price"], within=NonNegativeReals)
    m.crop_yield = Param(m.CROPS, initialize=nominal_data["Yield"], within=NonNegativeReals)
    m.min_required = Param(m.CROPS, initialize=nominal_data["Minimum Requirement"], within=NonNegativeReals)
    m.max_possible = Param(m.CROPS, initialize=nominal_data["Maximum Favorable Production"], within=NonNegativeReals)

    # Extract boolean parameters
    m.enforce_max_production = nominal_data["Enforce Max Production"].to_dict()
    m.enforce_min_requirement = nominal_data["Enforce Min Requirement"].to_dict()
    m.allow_purchases = nominal_data["Allow Purchases"].to_dict()

    # Stage 1 decision variables
    m.plant = Var(m.CROPS, within=NonNegativeReals)

    # Stage 2 decision variables
    m.buy = Var(m.CROPS, within=NonNegativeReals) # purchases
    m.sell_favorable = Var(m.CROPS, within=NonNegativeReals) # sales
    m.sell_unfavorable = Var(m.CROPS, within=NonNegativeReals) # sales

    for c in crops:
        # Disable purchases for crops with NaN prices
        if not m.allow_purchases[c]:
            m.buy[c].fix(0)

        # Enforce maximum production limits
        if m.enforce_max_production[c]:
            m.sell_favorable[c].setub(m.max_possible[c])

    # Total area constraint
    @m.Constraint()
    def total_area(m):
        return sum(m.plant[crop] for crop in m.CROPS) <= m.max_area

    # Constraint on the production of each crop
    @m.Constraint(m.CROPS)
    def crop_min_constraint(m, crop):
        if m.enforce_min_requirement[crop]:
            return m.plant[crop] * m.crop_yield[crop] + m.buy[crop] - m.sell_unfavorable[crop] - m.sell_favorable[crop] >= m.min_required[crop]
        else:
            return m.plant[crop] * m.crop_yield[crop] + m.buy[crop] - m.sell_unfavorable[crop] - m.sell_favorable[crop] >= 0

    # Maximize net profit
    @m.Expression()
    def net_profit(m):
        return -sum(m.plant[c] * m.cost[c] for c in crops) - sum(m.buy[c] * m.purchase_price[c] for c in crops) + sum(m.sell_favorable[c] * m.sell_price_favorable[c] + m.sell_unfavorable[c] * m.sell_price_unfavorable[c] for c in crops)

    if not skip_objective:
        m.obj = Objective(expr=m.net_profit, sense=maximize)

    return m

m = build_deterministic_model(nominal_data)

m.pprint()

1 Set Declarations
    CROPS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Wheat', 'Corn', 'Beats'}

7 Param Declarations
    cost : Size=3, Index=CROPS, Domain=NonNegativeReals, Default=None, Mutable=False
        Key   : Value
        Beats :   260
         Corn :   230
        Wheat :   150
    crop_yield : Size=3, Index=CROPS, Domain=NonNegativeReals, Default=None, Mutable=False
        Key   : Value
        Beats :  20.0
         Corn :   3.0
        Wheat :   2.5
    max_possible : Size=3, Index=CROPS, Domain=NonNegativeReals, Default=None, Mutable=False
        Key   : Value
        Beats : 6000.0
         Corn :    inf
        Wheat :    inf
    min_required : Size=3, Index=CROPS, Domain=NonNegativeReals, Default=None, Mutable=False
        Key   : Value
        Beats :   0.0
         Corn : 240.0
        Wheat : 200.0
    purchase_price : Size=3, Index=CROPS, Domain=NonNegativeReals, Default=None

In [11]:
solver = SolverFactory('ipopt')
solver.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 11, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.021295785903930664}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [12]:
def print_solution(m):

    # Display the results
    print("Planting decisions:")
    for crop in m.CROPS:
        print(f"{crop}: {round(m.plant[crop](),2)} acres")

    print("\nPurchase decisions:")
    for crop in m.CROPS:
        print(f"{crop}: {round(m.buy[crop](),2)} T")

    print("\nSales decisions (favorable price):")
    for crop in m.CROPS:
        print(f"{crop}: {round(m.sell_favorable[crop](),2)} T")

    print("\nSales decisions (unfavorable price):")
    for crop in m.CROPS:
        print(f"{crop}: {round(m.sell_unfavorable[crop](),2)} T")

print_solution(m)

Planting decisions:
Wheat: 120.0 acres
Corn: 80.0 acres
Beats: 300.0 acres

Purchase decisions:
Wheat: 0.0 T
Corn: 0.0 T
Beats: 0 T

Sales decisions (favorable price):
Wheat: 100.0 T
Corn: 0.0 T
Beats: 6000.0 T

Sales decisions (unfavorable price):
Wheat: 0.0 T
Corn: 0.0 T
Beats: 0.0 T


## Blocks

Now we will use blocks to construct the two-stage stochastic program.

### Prepare Input Data

In [13]:
scenarios = {}

yield_factors = [0.8, 1.0, 1.2]
scienario_names = ["BELOW", "AVERAGE", "ABOVE"]

for i, name in enumerate(scienario_names):
    # Copy the data
    scenario = nominal_data.copy()

    # Change the yield
    scenario["Yield"] = scenario["Yield"] * yield_factors[i]

    scenarios[name] = scenario

In [14]:
print(scenarios)

{'BELOW':        Yield  Planting Cost  Favorable Selling Price  \
index                                                  
Wheat    2.0            150                      170   
Corn     2.4            230                      150   
Beats   16.0            260                       36   

       Unfavorable Selling Price  Purchase Price  Minimum Requirement  \
index                                                                   
Wheat                        0.0           238.0                200.0   
Corn                         0.0           210.0                240.0   
Beats                       10.0             0.0                  0.0   

       Maximum Favorable Production  Enforce Max Production  \
index                                                         
Wheat                           inf                   False   
Corn                            inf                   False   
Beats                        6000.0                    True   

       Enforce Min Requirem

### Solve the Deterministic Model for Each Stage

In [15]:
for i, (key, value) in enumerate(scenarios.items()):
    print(f"*** Solving scenario {i} = {key} ***")
    
    m = build_deterministic_model(value)

    solver = SolverFactory('ipopt')
    solver.solve(m, tee=False)
    print_solution(m)
    print("\n")

*** Solving scenario 0 = BELOW ***
Planting decisions:
Wheat: 100.0 acres
Corn: 25.0 acres
Beats: 375.0 acres

Purchase decisions:
Wheat: 0.0 T
Corn: 180.0 T
Beats: 0 T

Sales decisions (favorable price):
Wheat: 0.0 T
Corn: 0.0 T
Beats: 6000.0 T

Sales decisions (unfavorable price):
Wheat: 0.0 T
Corn: 0.0 T
Beats: 0.0 T


*** Solving scenario 1 = AVERAGE ***
Planting decisions:
Wheat: 120.0 acres
Corn: 80.0 acres
Beats: 300.0 acres

Purchase decisions:
Wheat: 0.0 T
Corn: 0.0 T
Beats: 0 T

Sales decisions (favorable price):
Wheat: 100.0 T
Corn: 0.0 T
Beats: 6000.0 T

Sales decisions (unfavorable price):
Wheat: 0.0 T
Corn: 0.0 T
Beats: 0.0 T


*** Solving scenario 2 = ABOVE ***
Planting decisions:
Wheat: 183.33 acres
Corn: 66.67 acres
Beats: 250.0 acres

Purchase decisions:
Wheat: 0.0 T
Corn: 0.0 T
Beats: 0 T

Sales decisions (favorable price):
Wheat: 350.0 T
Corn: 0.0 T
Beats: 6000.0 T

Sales decisions (unfavorable price):
Wheat: 0.0 T
Corn: 0.0 T
Beats: 0.0 T




### Define the Two-Stage Stochastic Program Using Blocks

In [16]:
from pyomo.environ import Expression

def create_two_stage_stochastic_model(scenarios):

    # Model
    m = ConcreteModel()

    nominal_scenario_name = "AVERAGE"
    nominal_data = scenarios[nominal_scenario_name]

    # Sets
    crops = nominal_data.index.to_list()
    m.CROPS = Set(initialize=crops)

    m.max_area = 500

    # Stage 1 variables
    m.plant = Var(m.CROPS, within=NonNegativeReals, initialize=100, bounds=(0, m.max_area))

    # Stage 1 constraint
    @m.Constraint()
    def total_area(m):
        return sum(m.plant[crop] for crop in m.CROPS) <= m.max_area
    
    def second_stage_block(b, scenario_name):
        
        # print("name = ", scenario_name)

        # Grab data for this specific scenario
        s_data = scenarios[scenario_name]
        
        # Parameters
        b.cost = Param(m.CROPS, initialize=s_data["Planting Cost"], within=NonNegativeReals)
        b.sell_price_favorable = Param(m.CROPS, initialize=s_data["Favorable Selling Price"], within=NonNegativeReals)
        b.sell_price_unfavorable = Param(m.CROPS, initialize=s_data["Unfavorable Selling Price"], within=NonNegativeReals)
        b.purchase_price = Param(m.CROPS, initialize=s_data["Purchase Price"], within=NonNegativeReals)
        b.crop_yield = Param(m.CROPS, initialize=s_data["Yield"], within=NonNegativeReals)
        b.min_required = Param(m.CROPS, initialize=s_data["Minimum Requirement"], within=NonNegativeReals)
        b.max_possible = Param(m.CROPS, initialize=s_data["Maximum Favorable Production"], within=NonNegativeReals)

        # Extract boolean parameters
        b.enforce_max_production = s_data["Enforce Max Production"].to_dict()
        b.enforce_min_requirement = s_data["Enforce Min Requirement"].to_dict()
        b.allow_purchases = s_data["Allow Purchases"].to_dict()

        # Stage 2 decision variables
        b.buy = Var(m.CROPS, within=NonNegativeReals, initialize=0) # purchases
        b.sell_favorable = Var(m.CROPS, within=NonNegativeReals, initialize=0) # sales
        b.sell_unfavorable = Var(m.CROPS, within=NonNegativeReals, initialize=0) # sales

        for c in crops:
            # Disable purchases for crops with NaN prices
            if not b.allow_purchases[c]:
                b.buy[c].fix(0)

            # Enforce maximum production limits
            if b.enforce_max_production[c]:
                b.sell_favorable[c].setub(b.max_possible[c])

        # Constraint on the production of each crop
        @b.Constraint(m.CROPS)
        def crop_min_constraint(b, crop):
            if b.enforce_min_requirement[crop]:
                return m.plant[crop] * b.crop_yield[crop] + b.buy[crop] - b.sell_unfavorable[crop] - b.sell_favorable[crop] >= b.min_required[crop]
            else:
                return m.plant[crop] * b.crop_yield[crop] + b.buy[crop] - b.sell_unfavorable[crop] - b.sell_favorable[crop] >= 0

        @b.Expression()
        def scenario_net_profit(b):
            return -sum(m.plant[c] * b.cost[c] for c in crops) - sum(b.buy[c] * b.purchase_price[c] for c in crops) + sum(b.sell_favorable[c] * b.sell_price_favorable[c] + b.sell_unfavorable[c] * b.sell_price_unfavorable[c] for c in crops)

    # Create blocks for each scenario
    m.planning_scenarios = Block(scenarios.keys(), rule=second_stage_block)

    # Maximize net profot
    @m.Objective(sense=maximize)
    def obj(m):
        return sum(m.planning_scenarios[sc].scenario_net_profit for sc in scenarios.keys())/len(scenarios)
    
    return m

m = create_two_stage_stochastic_model(scenarios)
m.pprint()
    

1 Set Declarations
    CROPS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Wheat', 'Corn', 'Beats'}

1 Var Declarations
    plant : Size=3, Index=CROPS
        Key   : Lower : Value : Upper : Fixed : Stale : Domain
        Beats :     0 :   100 :   500 : False :  True : NonNegativeReals
         Corn :     0 :   100 :   500 : False :  True : NonNegativeReals
        Wheat :     0 :   100 :   500 : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : ((- (150*plant[Wheat] + 230*plant[Corn] + 260*plant[Beats]) - (238.0*planning_scenarios[BELOW].buy[Wheat] + 210.0*planning_scenarios[BELOW].buy[Corn] + 0.0*planning_scenarios[BELOW].buy[Beats]) + (170*planning_scenarios[BELOW].sell_favorable[Wheat] + 0.0*planning_scenarios[BELOW].sell_unfavorable[Wheat] + 150*planning_scenarios[BELOW

### Solve and Inspect Solution

In [17]:
solver = SolverFactory('ipopt')
solver.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 10, 'Number of variables': 27, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.02140498161315918}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [18]:
# Display the results
print("Planting decisions:")
for crop in m.CROPS:
    print(f"{crop}: {round(m.plant[crop](),2)} acres")

for i, name in enumerate(scenarios.keys()):
    print(f"\n *** Scenario {i} = {name} ***")
    

    print("\n\tPurchase decisions:")
    for crop in m.CROPS:
        print(f"\t{crop}: {round(m.planning_scenarios[name].buy[crop](),2)} T")

    print("\n\tSales decisions (favorable price):")
    for crop in m.CROPS:
        print(f"\t{crop}: {round(m.planning_scenarios[name].sell_favorable[crop](),2)} T")

    print("\n\tSales decisions (unfavorable price):")
    for crop in m.CROPS:
        print(f"\t{crop}: {round(m.planning_scenarios[name].sell_unfavorable[crop](),2)} T")

Planting decisions:
Wheat: 170.0 acres
Corn: 80.0 acres
Beats: 250.0 acres

 *** Scenario 0 = BELOW ***

	Purchase decisions:
	Wheat: 0.0 T
	Corn: 48.0 T
	Beats: 0 T

	Sales decisions (favorable price):
	Wheat: 140.0 T
	Corn: 0.0 T
	Beats: 4000.0 T

	Sales decisions (unfavorable price):
	Wheat: 0.0 T
	Corn: 0.0 T
	Beats: 0.0 T

 *** Scenario 1 = AVERAGE ***

	Purchase decisions:
	Wheat: 0.0 T
	Corn: 0.0 T
	Beats: 0 T

	Sales decisions (favorable price):
	Wheat: 225.0 T
	Corn: 0.0 T
	Beats: 5000.0 T

	Sales decisions (unfavorable price):
	Wheat: 0.0 T
	Corn: 0.0 T
	Beats: 0.0 T

 *** Scenario 2 = ABOVE ***

	Purchase decisions:
	Wheat: 0.0 T
	Corn: 0.0 T
	Beats: 0 T

	Sales decisions (favorable price):
	Wheat: 310.0 T
	Corn: 48.0 T
	Beats: 6000.0 T

	Sales decisions (unfavorable price):
	Wheat: 0.0 T
	Corn: 0.0 T
	Beats: 0.0 T


## Blocks with More Code Reuse

How can we compactly write our problem reusing our function for the deterministic (single scenario) case?

In [19]:
def create_two_stage_stochastic_model_better(scenarios):

    # Create a concrete model
    m = ConcreteModel()
    
    # Create a block for each scenario
    # This creates a copy of your deterministic model for each scenario
    m.planning_scenarios = Block(scenarios.keys(), rule=lambda b, s: build_deterministic_model(scenarios[s], m=b, skip_objective=True))

    # Enforce the same planting (stage 1) decisions for all scenarios
    m.SCENARIOS = Set(initialize=scenarios.keys())
    m.CROPS = Set(initialize=scenarios["AVERAGE"].index.to_list())

    @m.Constraint(m.SCENARIOS, m.CROPS)
    def enforce_same_planting(m, s, c):
        if s == "AVERAGE":
            return Constraint.Skip
        else:
            return m.planning_scenarios[s].plant[c] == m.planning_scenarios["AVERAGE"].plant[c]
    
    @m.Objective(sense=maximize)
    def obj(m):
        return sum(m.planning_scenarios[s].net_profit for s in scenarios.keys())/len(scenarios)

    return m

m = create_two_stage_stochastic_model_better(scenarios)
m.pprint()

2 Set Declarations
    CROPS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'Wheat', 'Corn', 'Beats'}
    SCENARIOS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'BELOW', 'AVERAGE', 'ABOVE'}

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : ((- (150*planning_scenarios[BELOW].plant[Wheat] + 230*planning_scenarios[BELOW].plant[Corn] + 260*planning_scenarios[BELOW].plant[Beats]) - (238.0*planning_scenarios[BELOW].buy[Wheat] + 210.0*planning_scenarios[BELOW].buy[Corn] + 0.0*planning_scenarios[BELOW].buy[Beats]) + (170*planning_scenarios[BELOW].sell_favorable[Wheat] + 0.0*planning_scenarios[BELOW].sell_unfavorable[Wheat] + 150*planning_scenarios[BELOW].sell_favorable[Corn] + 0.0*planning_scenarios[BELOW].sell_unfavorable[Corn] + 36*pla

In [20]:
solver = SolverFactory('ipopt')
solver.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 18, 'Number of variables': 33, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.023143768310546875}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [21]:
for i, name in enumerate(scenarios.keys()):
    print(f"*** Solving scenario {i} = {name} ***")
    print_solution(m.planning_scenarios[name])
    print("\n")

*** Solving scenario 0 = BELOW ***
Planting decisions:
Wheat: 170.0 acres
Corn: 80.0 acres
Beats: 250.0 acres

Purchase decisions:
Wheat: 0.0 T
Corn: 48.0 T
Beats: 0 T

Sales decisions (favorable price):
Wheat: 140.0 T
Corn: 0.0 T
Beats: 4000.0 T

Sales decisions (unfavorable price):
Wheat: 0.0 T
Corn: 0.0 T
Beats: 0.0 T


*** Solving scenario 1 = AVERAGE ***
Planting decisions:
Wheat: 170.0 acres
Corn: 80.0 acres
Beats: 250.0 acres

Purchase decisions:
Wheat: 0.0 T
Corn: 0.0 T
Beats: 0 T

Sales decisions (favorable price):
Wheat: 225.0 T
Corn: 0.0 T
Beats: 5000.0 T

Sales decisions (unfavorable price):
Wheat: 0.0 T
Corn: 0.0 T
Beats: 0.0 T


*** Solving scenario 2 = ABOVE ***
Planting decisions:
Wheat: 170.0 acres
Corn: 80.0 acres
Beats: 250.0 acres

Purchase decisions:
Wheat: 0.0 T
Corn: 0.0 T
Beats: 0 T

Sales decisions (favorable price):
Wheat: 310.0 T
Corn: 48.0 T
Beats: 6000.0 T

Sales decisions (unfavorable price):
Wheat: 0.0 T
Corn: 0.0 T
Beats: 0.0 T


